In [27]:
import torch
import sys
import gc
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WrapperConfig, SMOL_LM_135M

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
cfg = WrapperConfig(
    model_id_or_path=SMOL_LM_135M,
    max_samples=1000,
    n_epochs=10,
    data_module_choice="evol_codealpaca_dpo",
    notebook_mode=True,
    tuning_mode="sft",
    eval_data_mode="fixed"
)

wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2024-12-28 01:54:27.521 | INFO     | trl_wrapper.trainer_wrapper:init_model:136 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct


In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [29]:
wrapper.init_data_module()

2024-12-28 01:54:30.848 | INFO     | model.utils:__init__:104 - Cache dir: ../dataset_caches/evol_code_alpaca_data_module
2024-12-28 01:54:30.849 | INFO     | model.utils:setup:118 - Loading dataset for stage fit
2024-12-28 01:54:30.850 | INFO     | model.utils:setup:120 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/evol_code_alpaca_data_module
2024-12-28 01:54:30.851 | INFO     | dataset.code:load_dataset:110 - Loading dataset


2024-12-28 01:54:32.184 | WARNING  | model.utils:setup:165 - Columns in dataset: ['system', 'question', 'chosen', 'rejected', 'conversations'] do not match torch columns, not setting format


In [24]:
wrapper.data_module.train_dataset['conversations'][0]

[{'content': 'You are experienced in guiding both individual developers and teams, offering mentorship in personal growth, coding skills improvement, and collaborative development.',
  'role': 'system'},
 {'content': "The similarity measure between two attributes always falls in the range of $[0,1]$, why is this not true in the case of dissimilarity where the value falls in the range of $[0, \\infty)$?\nCan't the dissimilarity between them be thought of as the opposite of similarity in the sense that if they have a similarity of 0, then the dissimilarity will be 1 and vice versa?",
  'role': 'user'}]

In [30]:
wrapper.init_trainer()

2024-12-28 01:54:38.937 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:214 - Initializing trainer, run: run-12-28-1-54-390448-evol_codealpaca_dpo-sft-SmolLM2-135M-Instruct, project: codecontests-llama-3b
2024-12-28 01:54:38.937 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:217 - logprobs cache location: ../dataset_caches/evol_code_alpaca_data_module/b68b303b/ref_logprobs_cache peft config: False
2024-12-28 01:54:38.938 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:220 - WrapperConfig(model_id_or_path='HuggingFaceTB/SmolLM2-135M-Instruct', notebook_mode=True, max_sequence_length=1512, max_prompt_length=1024, max_eval_sample_length=1024, max_samples=1000, train_batch_size=4, eval_batch_size=2, gradient_accumulation_steps=1, gradient_checkpointing=True, root_dir=None, data_module_choice='evol_codealpaca_dpo', wandb_project_name='codecontests-llama-3b', n_epochs=10, max_eval_dataset_size=None, adapter_path=None, dpo_beta=0.1, learning_rate=5e-05, max_grad_norm=0.3, l

In [31]:
wrapper.train()

2024-12-28 01:54:41.147 | INFO     | trl_wrapper.trainer_wrapper:train:429 - Starting training.


Step,Training Loss,Validation Loss
0,No log,2.560204


KeyboardInterrupt: 